# Duckietown and ROS

In this notebook we will use what we have learned about ROS in the previous notebooks to start to control the duckiebot, either in simulation or on real hardware. 

## See what the Duckiebot Sees

As before, launch the noVNC browser by running following command in the terminal **of your laptop** (not the terminal in VSCode!) except this time with the `--sim` argument:

    dts code workbench --sim

Note 1: You will need to stop the previous with CTRL-C and then execute this with the `--sim` option.
Note 2: This may take some time to run the first time because it is downloaded the docker image of the Duckietown simulation environment. 

Once you open the noVNC browser, as before, you should see the same icons on the left hand side. Double-click on the one that says `RQT Image ...`, clicking on this opens the [rqt_image_view](https://wiki.ros.org/rqt_image_view) ROS utility.

## Create a New Package

Use the menu on the top-left corner of this editor, then `Terminal  ->  New Terminal` to open a terminal at the bottom of the page. Use the terminal to execute the commands introduced in this notebook. Feel free to experiment with this terminal as much as you need to get comfortable with these new commands.

## Introduction to msg

-   [msg](http://wiki.ros.org/msg): msg files are simple text files that describe the fields of a ROS message. They are used to generate source code for messages in different languages. msg files are stored in the msg directory of a package.

msgs are just simple text files with a field type and field name per line. The field types you can use are:

-   int8, int16, int32, int64 (plus uint\*)
-   float32, float64
-   string
-   time, duration
-   other msg files
-   variable-length array\[\] and fixed-length array\[C\]

There is also a special type in ROS: Header, the header contains a timestamp and coordinate frame information that are commonly used in ROS. You will frequently see the first line in a msg file have Header named header.

Here is an example of a msg that uses a Header, a string primitive, and two other msgs :

```
  Header header
  string child_frame_id
  geometry_msgs/PoseWithCovariance pose
  geometry_msgs/TwistWithCovariance twist
```



## Using msg

### Creating a msg

Let's define a new msg in the package that was created in the previous tutorial.  In the **EXPLORER** create a folder named `msg` inside the package `my_package`. Inside the `msg` folder create a file named `Num.msg` with the following content:

```
int64 num
```

The example .msg file above contains only 1 line. You can, of course, create a more complex file by adding multiple elements, one per line, like this:

```
string first_name
string last_name
uint8 age
uint32 score
```

There's one more step, though. We need to make sure that the msg files are turned into source code for C++, Python, and other languages:

Open package.xml, and make sure these two lines are in it and [uncommented](http://www.htmlhelp.com/reference/wilbur/misc/comment.html):

```
  <build_depend>message_generation</build_depend>
  <exec_depend>message_runtime</exec_depend>
```

Note that at build time, we need "message\_generation", while at runtime, we only need "message\_runtime".

Open CMakeLists.txt inside `./workspace/src/my_package/`.

Add the message\_generation dependency to the find\_package call which already exists in your CMakeLists.txt so that you can generate messages. You can do this by simply adding message\_generation to the list of COMPONENTS such that it looks like this:

```
# Do not just add this to your CMakeLists.txt, modify the existing text to add message_generation before the closing parenthesis
find_package(catkin REQUIRED COMPONENTS
   roscpp
   rospy
   std_msgs
   message_generation
)
```

You may notice that sometimes your project builds fine even if you did not call find\_package with all dependencies. This is because catkin combines all your projects into one, so if an earlier project calls find\_package, yours is configured with the same values. But forgetting the call means your project can easily break when built in isolation.

Also make sure you export the message runtime dependency.

```
catkin_package(
  ...
  CATKIN_DEPENDS message_runtime ...
  ...)
```

Add the following block of code:

```
add_message_files(
  FILES
  Num.msg
)
```

By adding the `.msg` files manually, we make sure that CMake knows when it has to reconfigure the project after you add other `.msg` files.

Now we must ensure the `generate_messages()` function is called by adding these lines:

```
generate_messages(
  DEPENDENCIES
  std_msgs
)
```


Now you're ready to generate source files from your msg definition.


## Re-build Workspace

We now proceed with compiling (or building) our workspace.
We can do so by running the following commands:

```{bash}
    cd workspace/
    catkin build
```

The build process should complete within a few seconds and the summary should report that `2` packages were built. Remember, the `catkin_tools_prebuild` package is present by default in every workspace.



## Using rosmsg

That's all you need to do to create a msg. Let's make sure that ROS can see it using the rosmsg show command.

Usage:

```
rosmsg show [message type]
```

You can run:

```
rosmsg show my_package/Num
```

You will see:

-   ```
    int64 num
    ```
    

In the previous example, the message type consists of two parts:

-   `my_package`: the package where the message is defined
    
-   `Num`: the name of the msg Num.
    

If you can't remember which Package a msg is in, you can leave out the package name. Try executing:

```
rosmsg show Num
```

You will see:

-   ```
    [my_package/Num]:
    int64 num
    ```
    